# Recreate Datasets with Twitter Ids

This notebook recreates the annotation dataset and the dataset with the model predictions.
Reminder: 
- This step has to be done because it isn't allowed to publish the dataset with the content of the tweets. Therfore the contents have to be retrieved with the tweet_id
- The datasets may be not 100% equal, as some Users or Tweets could have been suspended/removed in the meantime

In [2]:
import pandas as pd

In [171]:
%%capture
!pip install tweepy

# Recreate Annotation Dataset

In [199]:
import tweepy
from twitter_authentication import bearer_token, access_token, access_token_secret, api_key, api_key_secret

# Your Twitter API credentials - you can store them in a separate file, namely "twitter_authentication.py"
consumer_key = api_key
consumer_secret = api_key_secret
access_token = access_token
access_token_secret = access_token_secret

# Authenticate with the Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Example: Get the tweet with the given tweet ID
tweet_id = "1501140412630814727" # Replace with the actual tweet ID
tweet = api.get_status(tweet_id)

# Print the tweet text
print(tweet.text)

Wenn es ums Geld aus Gas-, Öl- &amp; Kohlegeschäften geht, hält die SPD-Bande von Gerds Schröder noch mit Putins Bande… https://t.co/nfMCWy8ANa


In [205]:
# Read in annotated dataset
# Note: some of the Tweets already have been suspendend, thats why there are just 1197 instead of 1250 samples
df = pd.read_csv("annotation_dataset_tweet_ids.csv")
df

,Unnamed: 0,tweet_id,majority_vote
0,0,1501140412630814727,1.0
1,3,1496911493929050119,0.0
2,4,1502408036350058501,1.0
3,5,1499713163658473477,0.0
4,6,1497980730844098565,1.0
...,...,...,...
1192,1599,1499190024822210562,1.0
1193,1601,1489513947795529734,1.0
1194,1603,1508778469571837961,2.0
1195,1604,1498275319383478277,1.0


In [207]:
# Function to retrieve tweets by ID
def get_tweet_info(tweet_id):
    try:
        tweet = api.get_status(tweet_id, tweet_mode='extended')
        res = {
            'tweet_id': tweet.id_str,
            'created_at': tweet.created_at,
            'author_id': tweet.user.id_str,
            'text': tweet.full_text,
            'retweets': tweet.retweet_count,
            'likes': tweet.favorite_count,
            'mentions': [user['screen_name'] for user in tweet.entities['user_mentions']]
        }
        
    except tweepy.errors.TweepyException as e:
        print(f"Error: {e}")
        return None
    
df['tweet_text'] = df['tweet_id'].apply(get_tweet_info)

Rate limit reached. Sleeping for: 331


Error: 403 Forbidden
63 - User has been suspended.


In [223]:
# Extract the resulting dictionary into separate columns
# The Annotation Dataset is ready to use!
df[['tweet_id', 'created_at', 'author_id', 'text', 'retweets', 'likes', 'mentions']] = df['tweet_text'].apply(lambda x: pd.Series([x['tweet_id'], x['created_at'], x['author_id'], x['text'],x['retweets'], x['likes'],x['mentions']]if x is not None else [None, None, None,None,None, None, None]))
df                                                                                                                          

,Unnamed: 0,tweet_id,majority_vote,tweet_text,created_at,author_id,text,retweets,likes,mentions
0,0,1501140412630814727,1.0,"{'tweet_id': '1501140412630814727', 'created_a...",2022-03-08 10:18:58+00:00,2362449115,"Wenn es ums Geld aus Gas-, Öl- &amp; Kohlegesc...",7.0,16.0,"[OlafScholz, ABaerbock, c_lindner]"
1,3,1496911493929050119,0.0,"{'tweet_id': '1496911493929050119', 'created_a...",2022-02-24 18:14:46+00:00,1021088482066870273,@Bundeskanzler @OlafScholz Deutschland muss ha...,0.0,0.0,"[Bundeskanzler, OlafScholz]"
2,4,1502408036350058501,1.0,"{'tweet_id': '1502408036350058501', 'created_a...",2022-03-11 22:16:03+00:00,864527983,@spdde @larsklingbeil Ihr seid und bleibt Apol...,0.0,0.0,"[spdde, larsklingbeil]"
3,5,1499713163658473477,0.0,"{'tweet_id': '1499713163658473477', 'created_a...",2022-03-04 11:47:36+00:00,1219288469169737730,@SWagenknecht Die Deutschen werden bis heute f...,0.0,0.0,[SWagenknecht]
4,6,1497980730844098565,1.0,"{'tweet_id': '1497980730844098565', 'created_a...",2022-02-27 17:03:32+00:00,2725063516,@MarcoBuschmann @DasRechtsportal Mann!! Bushma...,5.0,15.0,"[MarcoBuschmann, DasRechtsportal]"
...,...,...,...,...,...,...,...,...,...,...
1192,1599,1499190024822210562,1.0,"{'tweet_id': '1499190024822210562', 'created_a...",2022-03-03 01:08:50+00:00,2350766257,@archangel_riku @OliverKarst @OlafScholz @ABae...,0.0,0.0,"[OliverKarst, OlafScholz, ABaerbock]"
1193,1601,1489513947795529734,1.0,"{'tweet_id': '1489513947795529734', 'created_a...",2022-02-04 08:19:33+00:00,1392835700706267137,@IreneMihalic @GrueneBundestag @janoschdahmen ...,0.0,2.0,"[IreneMihalic, GrueneBundestag, janoschdahmen]"
1194,1603,1508778469571837961,2.0,"{'tweet_id': '1508778469571837961', 'created_a...",2022-03-29 12:09:53+00:00,1199312834330578945,@SWagenknecht Hat #BuReg noch alles im Griff?\...,2.0,23.0,[SWagenknecht]
1195,1604,1498275319383478277,1.0,"{'tweet_id': '1498275319383478277', 'created_a...",2022-02-28 12:34:07+00:00,1154831779657859072,@StBrandner So früh schon betrunken? Meine Fre...,0.0,0.0,[StBrandner]


In [ ]:
# Save dataset
#df.to_csv("annotated_dataset.csv")

# Recreating Dataset with model predictions

In [3]:
# Load the dataframe with the model predictions. Note: Some tweets already have been suspended, thats why the Lenght of the dataframes may differ.
df = pd.read_csv("model_predictions_dataset_tweet_ids.csv")
df

,Unnamed: 0,tweet_id,model_predictions,Partei
0,0,1509679863984275460,0,Die Linke
1,1,1509679798100140038,0,Die Linke
2,2,1509679601269751812,0,Die Linke
3,3,1509678314490892289,0,Die Linke
4,4,1509678301924765704,0,Die Linke
...,...,...,...,...
1775245,1775245,1503352624468398082,0,CSU/CDU
1775246,1775246,1503352471036600320,0,CSU/CDU
1775247,1775247,1503352431194824712,0,CSU/CDU
1775248,1775248,1503352286512308226,0,CSU/CDU


In [ ]:
df['tweet_text'] = df['tweet_id'].apply(get_tweet_info)

In [ ]:
# Extract the resulting dictionary into separate columns
# The Annotation Dataset is ready to use!
df[['tweet_id', 'created_at', 'author_id', 'text', 'retweets', 'likes', 'mentions']] = df['tweet_text'].apply(lambda x: pd.Series([x['tweet_id'], x['created_at'], x['author_id'], x['text'],x['retweets'], x['likes'],x['mentions']]if x is not None else [None, None, None,None,None, None, None]))
df   